In [1]:

import pandas as pd

In [3]:
preds = pd.read_csv("holdout_unfiltered_predictions_step_1536_tta_None.csv")
preds.describe()

,vessel_length_m,detect_scene_row,detect_scene_column,objectness_p,is_vessel_p,is_fishing_p,objectness_threshold
count,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.0
mean,24.411436,11511.197018,14396.790380,0.442755,0.810935,0.148028,0.3
std,18.015006,5220.281014,6182.476614,0.143879,0.218972,0.277612,0.0
min,13.742188,733.000000,602.000000,0.300049,0.034180,0.000010,0.3
25%,16.468750,7028.250000,9770.000000,0.333252,0.793945,0.000660,0.3
50%,18.109375,11391.000000,14390.500000,0.383789,0.913086,0.009087,0.3
75%,22.750000,15796.000000,18384.750000,0.514160,0.937012,0.111084,0.3
max,165.250000,23401.000000,28681.000000,0.883301,0.966309,0.942383,0.3


In [6]:
from xview3 import XView3DataModule

data_dir = "f:/datasets/xview3"
data = XView3DataModule(data_dir)

_, _, holdout_df, shore_root = data.train_val_split(
    splitter={"name":"precomputed", "split_csv": "../configs/dataset/valid_only_split.csv"},
    fold=0,
    num_folds=4,
)
holdout_scenes = list(holdout_df.scene_path.unique())

In [14]:
objectness_threshold, vessel_threshold, fishing_threshold = 0.55, 0.45, 0.4

In [15]:
from xview3 import apply_thresholds, filter_low_confidence_objects
from xview3.metric_mp import score_mp

score_mp(
    apply_thresholds(preds, objectness_threshold, vessel_threshold, fishing_threshold),
    filter_low_confidence_objects(holdout_df),
    shore_root, workers=6
)

{'loc_fscore': 0.6164533820840951,
 'loc_fscore_shore': 0.24165707710011505,
 'vessel_fscore': 0.9131832797427653,
 'fishing_fscore': 0.8676056338028169,
 'length_acc': 0.6769496882939042,
 'aggregate': 0.4561009955899208,
 'loc_precision': 0.6966942148760331,
 'loc_recall': 0.5527868852459017,
 'vessel_precision': 0.8580060422960725,
 'vessel_recall': 0.9759450171821306,
 'fishing_precision': 0.88,
 'fishing_recall': 0.8555555555555555}

In [17]:
def apply_thresholds_smart(
    df: pd.DataFrame, objectness_threshold: float, is_vessel_threshold: float, is_fishing_threshold: float
) -> pd.DataFrame:
    df = df[df.objectness_p >= objectness_threshold].copy().reset_index(drop=True)

    df["is_fishing"] = (df["is_fishing_p"] >= is_fishing_threshold) & (df["is_vessel_p"] >= is_vessel_threshold)
    df["is_vessel"] = (df["is_vessel_p"] >= is_vessel_threshold) | (df["is_fishing_p"] >= is_fishing_threshold)

    #df["is_fishing"] = (df["is_fishing_p"] >= is_fishing_threshold) #& (df["is_vessel_p"] >= is_vessel_threshold)
    #df["is_vessel"] = (df["is_vessel_p"] >= is_vessel_threshold) | (df["is_fishing_p"] >= is_fishing_threshold)
    return df

score_mp(
    apply_thresholds_smart(preds, objectness_threshold, vessel_threshold, fishing_threshold),
    filter_low_confidence_objects(holdout_df),
    shore_root, workers=6
)

{'loc_fscore': 0.6164533820840951,
 'loc_fscore_shore': 0.24165707710011505,
 'vessel_fscore': 0.9124497991967871,
 'fishing_fscore': 0.8676056338028169,
 'length_acc': 0.6769496882939043,
 'aggregate': 0.45601056427726866,
 'loc_precision': 0.6966942148760331,
 'loc_recall': 0.5527868852459017,
 'vessel_precision': 0.8567119155354449,
 'vessel_recall': 0.9759450171821306,
 'fishing_precision': 0.88,
 'fishing_recall': 0.8555555555555555}